<a href="https://colab.research.google.com/github/opassos/LetsCode/blob/main/Let's_Code_Projeto_DataViz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# https://www.kaggle.com/c/kaggle-survey-2021/data
kaggle_df = pd.read_csv('/content/drive/MyDrive/kaggle_survey/kaggle2021.csv')
desc = kaggle_df.iloc[0]
kaggle_df = kaggle_df.iloc[1:]
kaggle_df.head(1)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,195,201,285,286,287,288,289,290,291,292) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Time from Start to Finish (seconds),Q1,Q2,Q3,Q4,Q5,Q6,Q7_Part_1,Q7_Part_2,Q7_Part_3,Q7_Part_4,Q7_Part_5,Q7_Part_6,Q7_Part_7,Q7_Part_8,Q7_Part_9,Q7_Part_10,Q7_Part_11,Q7_Part_12,Q7_OTHER,Q8,Q9_Part_1,Q9_Part_2,Q9_Part_3,Q9_Part_4,Q9_Part_5,Q9_Part_6,Q9_Part_7,Q9_Part_8,Q9_Part_9,Q9_Part_10,Q9_Part_11,Q9_Part_12,Q9_OTHER,Q10_Part_1,Q10_Part_2,Q10_Part_3,Q10_Part_4,Q10_Part_5,Q10_Part_6,...,Q34_B_Part_6,Q34_B_Part_7,Q34_B_Part_8,Q34_B_Part_9,Q34_B_Part_10,Q34_B_Part_11,Q34_B_Part_12,Q34_B_Part_13,Q34_B_Part_14,Q34_B_Part_15,Q34_B_Part_16,Q34_B_OTHER,Q36_B_Part_1,Q36_B_Part_2,Q36_B_Part_3,Q36_B_Part_4,Q36_B_Part_5,Q36_B_Part_6,Q36_B_Part_7,Q36_B_OTHER,Q37_B_Part_1,Q37_B_Part_2,Q37_B_Part_3,Q37_B_Part_4,Q37_B_Part_5,Q37_B_Part_6,Q37_B_Part_7,Q37_B_OTHER,Q38_B_Part_1,Q38_B_Part_2,Q38_B_Part_3,Q38_B_Part_4,Q38_B_Part_5,Q38_B_Part_6,Q38_B_Part_7,Q38_B_Part_8,Q38_B_Part_9,Q38_B_Part_10,Q38_B_Part_11,Q38_B_OTHER
1,910,50-54,Man,India,Bachelor’s degree,Other,5-10 years,Python,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Python,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vim / Emacs,NaN,NaN,NaN,NaN,NaN,Colab Notebooks,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = kaggle_df[['Q2', 'Q3', 'Q5', 'Q8', 'Q15']].rename(columns = { 
    'Q2': 'Genero',
    'Q3': 'Pais',
    'Q5': 'Cargo',
    'Q8': 'Linguagem',
    'Q15': 'Experiencia'
})
other_countries = df.groupby('Pais').size().sort_values().iloc[:-9].index.values
other_languages = df.groupby('Linguagem').size().sort_values().iloc[:-4].index.values
other_jobs = np.append(df.groupby('Cargo').size().sort_values().iloc[:-5].index.values, 'Other')
df = df.replace({'Pais':{k:'Other Countries' for k in other_countries},
                 'Linguagem': {k:'Other Languages' for k in other_languages},
                 'Cargo': {k:'Other Jobs' for k in other_jobs},
                 'Experiencia': {'I do not use machine learning methods': 'No Experience'}})
df = df.dropna()
df.describe()

,Genero,Pais,Cargo,Linguagem,Experiencia
count,23754,23754,23754,23754,23754
unique,5,10,5,5,9
top,Man,Other Countries,Other Jobs,Python,Under 1 year
freq,18994,9322,9735,19281,9163


In [19]:
def get_data(df, input, output):
    data = (df
            .rename(columns = {input:'input', output:'output'})
            .groupby(['input', 'output'])
            .size()
            .to_frame('size')
            .reset_index()
    )
    return data

In [41]:
import plotly.graph_objects as go

def sankey_plot(df, columns, color_at, color):
    labels = np.concatenate(df.apply(lambda x: x.unique(), axis = 0).values)
    labels = {v:k for k,v in enumerate(labels)}
    _df = df.replace(labels)
    data = []

    for i in range(len(columns) - 1):
        data.append(get_data(_df, columns[i], columns[i + 1]))
    data = pd.concat(data)
    data['color'] = 'lightgray'

    for c_at, c in zip(color_at, color):
        data.loc[(data[['input', 'output']] == labels[c_at]).any(axis = 1), 'color'] = c

    fig = go.Figure(data=[go.Sankey(
        node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = np.array(list(labels.keys()))[list(labels.values())],
        color = "lightgray"
        ),
        link = dict(
        source = data['input'], # indices correspond to labels, eg A1, A2, A1, B1, ...
        target = data['output'],
        value = data['size'],
        color = data['color']
    ))])

    fig.update_layout()
    fig.show()

In [50]:
sankey_plot(df, ['Genero', 'Pais', 'Linguagem', 'Cargo', 'Experiencia'], 
            ['Python', 'Man', 'Brazil', 'Under 1 year', '20 or more years'], 
            ['orange', 'blue', 'green', 'lightblue', 'red'])